In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from scipy import stats
from sklearn.manifold import TSNE
from sklearn.impute import KNNImputer

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Drop missing data
# train_df.dropna(inplace=True)

# Drop outliers
# z_scores = np.abs(stats.zscore(train_df.select_dtypes(include=[np.number])))
# train_df = train_df[(z_scores < 3).all(axis=1)]

# train_df['ShootingEfficiency'] = train_df['FG_PCT'] * train_df['FG3_PCT']
# test_df['ShootingEfficiency'] = test_df['FG_PCT'] * test_df['FG3_PCT']

# train_df['TotalAttempts'] = train_df['FGA'] + train_df['FG3A'] + train_df['FTA']
# test_df['TotalAttempts'] = test_df['FGA'] + test_df['FG3A'] + test_df['FTA']

X = train_df.drop(['position', 'SEASON_ID', 'TEAM_ID', 'GS', 'GP', 'MIN'], axis=1)
y = train_df['position']
test_df = test_df.drop(['ID', 'SEASON_ID', 'TEAM_ID'], axis=1)

# fill missing data
imputer = KNNImputer(n_neighbors=5)
imputer.fit(X.dropna())
X = imputer.transform(X)

# TSNE
tsne = TSNE(n_components=3, random_state=42)

X_tsne = tsne.fit_transform(X)
X_tsne_df = pd.DataFrame(X_tsne, columns=['TSNE1', 'TSNE2', 'TSNE3'])
X = pd.DataFrame(X, columns=train_df.drop(['position', 'SEASON_ID', 'TEAM_ID', 'GS', 'GP', 'MIN'], axis=1).columns)
X = pd.concat([X, X_tsne_df], axis=1)

test_df_tsne = tsne.fit_transform(test_df)
test_df_tsne_df = pd.DataFrame(test_df_tsne, columns=['TSNE1', 'TSNE2', 'TSNE3'])
test_df = pd.DataFrame(test_df, columns=test_df.columns)
test_df = pd.concat([test_df, test_df_tsne_df], axis=1)

# standardize data
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# minmax scale data
# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

# encode target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# split data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Model
model = KNeighborsClassifier(n_neighbors=12, weights='distance', algorithm='auto')
# model = model.fit(X_train, y_train)
model = model.fit(X, y_encoded)
# f1 score
# y_pred = model.predict(X_val)
# f1 = f1_score(y_val, y_pred, average='weighted')
# print(f"f1 score: {f1}")

c:\Users\carot\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\carot\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [3]:
X_test = test_df
# X_test = scaler.transform(X_test)
y_test = model.predict(X_test)
y_test = label_encoder.inverse_transform(y_test)

result_df = pd.DataFrame({'ID': np.arange(1, len(y_test) + 1), 'position': y_test})
result_df.to_csv('result.csv', index=False)